In [ ]:
### #code adapted from https://www.plus2net.com/python/tkinter-df-search.php and https://www.pythontutorial.net/tkinter/tkinter-open-file-dialog/ and is modified

import pandas as pd #to read csv file
import tkinter as tk
from tkinter import ttk  # for treeview
from tkinter import filedialog as fd # for opening files
from tkinter.messagebox import showinfo # for showing filepath

#load the file
df = pd.read_csv('/Users/kaniyah/Downloads/test.csv') # create DataFrame

df.keys()

# creating a list of column names by calling the .columns
list_of_column_names = list(df.columns)
def lowercaseSearch():
#convert all search words in columns to lowercase if the column is not empty or contains NaN
    if df['Panel'].dropna().empty == True:
        pass
    else:
        df['Panel'] = df['Panel'].str.lower()
    if df['Input Author'].dropna().empty == True:
        pass
    else:
        df['Input Author'] = df['Input Author'].str.lower()
    if df['Bibcode'].dropna().empty == True:
        pass
    else:
        df['Bibcode'] = df['Bibcode'].str.lower()
    if df['Input Institution'].dropna().empty == True:
        pass
    else:
        df['Input Institution'] = df['Input Institution'].str.lower()
    if df['Abstract'].dropna().empty == True:
        pass
    else:
        df['Abstract'] = df['Abstract'].str.lower()
lowercaseSearch()

my_w = tk.Tk()
#my_w.resizable(width = 1200, height = 350)
my_w.title("Reviewer Extractor")  # Adding a title
my_w.maxsize(width=1440, height=550)

l1 = tk.Label(my_w, text="Enter your word(s):", width=15, font=18)  # added one Label
l1.grid(row=1, column=1, pady=10)

e1 = tk.Entry(my_w, width=35, bg="light gray", font=18)  # added one Entry box
e1.grid(row=1, column=2)

b1 = tk.Button(my_w, text="Search", width=7, font=18, command=lambda: my_search())
b1.grid(row=1, column=3)

loadb = tk.Button(my_w, text="Load csv file", width=12, font=18, command=lambda: load_csv())
loadb.grid(row=2, column=1)

# label frame
lf = ttk.LabelFrame(my_w, text='Select a column to search')
lf.grid(column=2, row=2, padx=20, pady=20)

alignment_var = tk.StringVar()
alignments = ('Author', 'Institution', 'Bibcode', 'Panels', 'Abstract')

# create radio buttons and place them on the label frame

grid_column = 0
for alignment in alignments:
    # create a radio button
    radio = ttk.Radiobutton(lf, text=alignment, value=alignment, variable=alignment_var)
    radio.grid(column=grid_column, row=0, ipadx=10, ipady=10)
    # grid column
    grid_column += 1

def load_csv():
    filetypes = (
        ('csv files', '*.csv'),
        ('All files', '*.*')
    )

    filename = fd.askopenfilename(
        title='Open a file',
        initialdir='/',
        filetypes=filetypes)

    showinfo(
        title='Selected File',
        message=filename
    )
    global df
    df = pd.read_csv(filename) #creates a new datafame from filename path
    print(df)


def checkbox_search():
    
    #checks which column has been selected in order to search in it
    global col 
    if alignment_var.get() == "Author":
        col = "True Author"
    elif alignment_var.get() == "Bibcode":
        col = "Bibcode"
    elif alignment_var.get() == "Institution":
        col = "True Institution"
    elif alignment_var.get() == "Panels":
        col = "Panels"
    elif alignment_var.get() == "Abstract":
        col = "Abstract"
    else:
        showinfo(
        title='Error',
        message='Please select a column'
    )

def my_search():
    checkbox_search() #determines which column is selected
    print("column searched: ", col)
   
    l1 = list(df)  # List of column names as headers
    query = e1.get().strip() # get user entered string
    print("word searched:", query)
    global df2
    if df[col].dropna().empty == True:
        df2 = df[col].where(pd.notnull(df[col]), None) #if column contains is empty and contains NaN, turn it into None
    else:
        df2 = df[df[col].str.contains(query) == True]  # combine all conditions using | operator
    r_set = df2.to_numpy().tolist()  # Create list of list using rows
    trv = ttk.Treeview(my_w, selectmode="browse")  # selectmode="browse" or "extended"
    trv.grid(row=4, column=1, columnspan=3, padx=10, pady=20)  #
    trv["height"] = 15  # Number of rows to display, default is 10
    trv["show"] = "headings"
    # column identifiers
    trv["columns"] = l1
    for i in l1:
        #set columns width to width of window divided by the number of columns
        trv.column(i, width=int((1440/len(l1))-2), anchor="c")
        # Headings of respective columns
        trv.heading(i, text=i)
    for dt in r_set:
        v = [r for r in dt]  # creating a list from each row
        print(v[0])
        try:
            trv.insert("", "end", iid=v[0], values=v)  # adding row
        except:
            continue
        #if id value already in chart, find a way to bypass it
    
        #adds save button, and saves the csv file to what the user wants as a csv
    b2 = tk.Button(my_w, text="Save CSV", bg="blue", width=10, font=18, command=lambda: save_to_csv())
    b2.grid(row=2, column=3)
    
def save_to_csv():
    #if the dataframe is not empty, allow it to be saved
    if df2 is not None:
        file_path = fd.asksaveasfilename(defaultextension=".csv", filetypes=[("CSV files", "*.csv")])

        if file_path:
            df2.to_csv(file_path, index=False)
            print(f"Data saved to {file_path}")
    else:
        print("DataFrame is not available, you must create it first.")
        
my_w.mainloop()
 